In [36]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [37]:
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
from dash import html, dash_table
from dash.dependencies import Input, Output
import json

In [38]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [39]:
import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point, Polygon
import geopy
from scipy.stats import pearsonr

In [40]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [41]:
complete_data = pd.read_csv('complete_data.csv')

In [42]:

elbow_results = pd.read_csv("knn_elbow_results.csv")
importance_df = pd.read_csv("knn_feature_importance.csv")

with open("knn_summary.json") as f:
    knn_summary = json.load(f)

cm_normalized = np.load("knn_confusion_matrix.npy")

In [43]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split


complete_data = complete_data.dropna(subset=['Crime Count'])
complete_data[['prcp', 'wspd']] = complete_data[['prcp', 'wspd']].fillna(0)
complete_data = complete_data.dropna(subset=['tavg'])
complete_data['Crime Count'] = (
    complete_data['Crime Count']
    .astype(str)
    .str.replace(r'[^0-9.\-]', '', regex=True)  # remove any commas or stray symbols
    .replace('', pd.NA)
    .astype(float)
)



cols_to_convert = [ 'tavg', 'prcp', 'wspd', 'Crime Count']
complete_data[cols_to_convert] = complete_data[cols_to_convert].apply(pd.to_numeric, errors='coerce')


complete_data = complete_data.dropna(subset=cols_to_convert)

X = complete_data[['tavg', 'prcp', 'wspd']]
y = complete_data['Crime Count']


X = sm.add_constant(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.astype('float64')
y_train = y_train.astype('float64')


model = sm.OLS(y_train, X_train).fit()
summary_text = model.summary().as_text()


fig_scatter = px.scatter(
    complete_data,
    x="tavg",
    y="Crime Count",
    trendline="ols",
    title="Crime Count vs Avg Temperature"
)

model = sm.OLS(y_train, X_train).fit()
summary_text = model.summary().as_text()


fig_scatter_avg_temp = px.scatter(
    complete_data,
    x="tavg",
    y="Crime Count",
    trendline="ols",
    title="Crime Count vs Avg Temperature"
)

fig_scatter_prcp = px.scatter(
    complete_data,
    x="prcp",
    y="Crime Count",
    trendline="ols",
    title="Crime Count vs Precipitation"
)

fig_scatter_wind = px.scatter(
    complete_data,
    x="wspd",
    y="Crime Count",
    trendline="ols",
    title="Crime Count vs Wind Speed"
)


import plotly.express as px

counts = complete_data.groupby(['Offense_cat', 'cluster']).size().reset_index(name='count')
counts['percent'] = counts.groupby('Offense_cat')['count'].transform(lambda x: x / x.sum() * 100)

fig_bar = px.bar(
    counts,
    x="Offense_cat",
    y="percent",
    color="cluster",
    title="Percent Stacked Bar Chart of Offense_cat by Cluster",
    barmode="stack"
)

fig_box_tavg = px.box(
    complete_data, x="cluster", y="tavg",
    title="Boxplot of tavg by Cluster (ordered)",
    category_orders={"cluster": [0,1,2,3,4,5,6]}
)

fig_box_hour = px.box(
    complete_data, x="cluster", y="HourReported",
    title="Boxplot of HourReported by Cluster (ordered)",
    category_orders={"cluster": [0,1,2,3,4,5,6]}
)

fig_box_prcp = px.box(
    complete_data, x="cluster", y="prcp",
    title="Boxplot of prcp by Cluster (ordered)",
    category_orders={"cluster": [0,1,2,3,4,5,6]}
)


import statsmodels.stats.multicomp as mc

clean_data = complete_data.dropna(subset=['tavg'])
comp = mc.MultiComparison(clean_data['tavg'], clean_data['cluster'])
tukey_result = comp.tukeyhsd()
tukey_summary = tukey_result.summary().as_text()

In [44]:
knn_best_df=pd.read_csv("knn_best.csv")

In [45]:
knn_best=knn_best_df.T

In [46]:
knn_best

,0,1,2
Best Parameters,21,1,distance
Best CV Accuracy,0.295647,0.295647,0.295647
Test Set Accuracy,0.296559,0.296559,0.296559


## Logistic Regression

In [47]:
crime_df=pd.read_csv("CrimeData_with_Neighborhood_Info.csv")


# BELOW IS ALL PREP FOR LOGISTIC REGRESSION

# Develop a category map to assign each offense to a broader topical category to simplify and generalize analysis

category_map = {
    'Assault & Violent Crime': ['assault', 'battery', 'homicide', 'murder', 'rape', 'sex offense', 'threats', 'robbery','harassment','stalking','kidnap','prowler','extortion'],
    'Theft & Larceny': ['larceny', 'shoplifting', 'stolen', 'theft', 'embezzlement','burglary'],
    'Property Damage': ['vandalism', 'graffiti', 'damage', 'arson'],
    'Narcotics/Drug Offenses': ['narcotics', 'drug', 'marijuana', 'possession', 'distribute','driving under the influence','towed vehicle'],
    'Motor Vehicle/Traffic': ['traffic', 'dui', 'dwi', 'reckless', 'unlicensed', 'accident','hit and run'],
    'Weapon Offenses': ['weapon', 'firearm', 'gun', 'shots fired', 'illegal hunting'],
    'Public Order/Disorderly': ['disorderly', 'public intoxication', 'trespassing', 'loitering', 'noise','domestic disturbance','disturbarance - non domestic','trespass','prostitution'],
    'Fraud & Financial': ['fraud', 'forgery', 'counterfeit', 'scam', 'financial','false report of crime'],
    'Mental Health/Welfare': ['mental', 'tdo', 'eco', 'suicide', 'assist citizen','missing person','runaway','crisis assessment','juvenile investigation']
}

# Implement the category map to the crime data

def assign_category(offense_type):
    """
    Checks the offense type against defined keywords and assigns a category.
    Returns 'Other' if no match is found.
    """
    # Convert to lowercase for case-insensitive matching
    offense_lower = offense_type.lower()
    
    # Iterate through categories and their keywords
    for category, keywords in category_map.items():
        if any(keyword in offense_lower for keyword in keywords):
            return category
    
    return 'Other'



crime_df['Offense_cat'] = crime_df['Offense'].apply(assign_category)
crime_df["Binary_Violence"]=crime_df["Offense_cat"]
crime_df["Binary_Violence"]=crime_df["Binary_Violence"].replace({'Mental Health/Welfare':0,
 'Weapon Offenses':0,
 'Assault & Violent Crime':1,
 'Theft & Larceny':0,
 'Public Order/Disorderly':0,
 'Other':0,
 'Narcotics/Drug Offenses':0,
 'Motor Vehicle/Traffic':0,
 'Fraud & Financial':0,
 'Property Damage':0})

# Set Date to DateTime datatype
crime_df["Date"]=crime_df["DateReport"].astype(str)

crime_df["Date"]=pd.to_datetime(crime_df["Date"], yearfirst=True)
crime_df["Month"]=crime_df["Date"].dt.month
crime_df["Year"]=crime_df["Date"].dt.year
crime_df["Day"]=crime_df["Date"].dt.day
crime_df["Hour"]=crime_df["Date"].dt.hour
subset=crime_df[["Year", "Month", "Day", "Hour", "NeighborhoodInfoNAME", "NeighborhoodInfoPOPULATION", "Binary_Violence"]].dropna()


/var/folders/09/89r30_ms6xdcd7znk61plqgr0000gn/T/ipykernel_14833/18039697.py:41: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [69]:
# BUILD LOGISTIC REGRESSION MODEL

X_log=subset[["Year", "Month", "Day", "Hour", "NeighborhoodInfoNAME", "NeighborhoodInfoPOPULATION"]]
y=subset["Binary_Violence"]

cats=["NeighborhoodInfoNAME"]
nums=["Year", "Month", "Day", "Hour", "NeighborhoodInfoPOPULATION"]

X_train, X_test, y_train, y_test= train_test_split(X_log, y, test_size=.2, random_state=42, stratify=y)

preprocess=ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), nums),
        ("cat", OneHotEncoder(drop="first"), cats)
    ]
)

logreg=LogisticRegression(max_iter=300000, random_state=42, class_weight="balanced")

pipe= Pipeline(steps=[("preprocess", preprocess),
                      ("model", logreg)])

pipe.fit(X_train,y_train)

print(pipe.score(X_test,y_test))

# INTERPRET THE MODEL RESULTS

feature_names = pipe.named_steps["preprocess"].get_feature_names_out()
feature_names = np.array(feature_names).flatten()

coefs = pipe.named_steps["model"].coef_[0]

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coef": coefs
}).sort_values(by="coef", key=abs, ascending=False)
feature_names = pd.Series(feature_names)
clean_names = (
    feature_names.str.replace('cat__NeighborhoodInfoNAME_', 'Neighborhood Name = ', regex=False))
coef_df["feature"]=clean_names

coef_df["odds_ratio"] = coef_df["coef"].apply(lambda x: np.exp(x))

logreg_interpret_table=coef_df.sort_values("odds_ratio", ascending=False)

p=pipe.predict_proba(X_test)[:,1]
y_hat=(p>=0.62).astype(int)
results= pd.DataFrame({"Actual Violent Crime":y_test, 
                       "Predicted Violent Crime": p.round(3),
                       "Predicted_label":y_hat})
results["Accuracy"] = (
    results["Actual Violent Crime"] == results["Predicted_label"]
).map({True: "Accurate", False: "Incorrect"})

results["Accuracy"].value_counts()

0.5814849044978435


Accuracy
Accurate     35058
Incorrect    10386
Name: count, dtype: int64

In [49]:
# create a dictionary to reference so user doesn't input the neighborhood population

pop_lookup = (
    subset[["NeighborhoodInfoNAME", "NeighborhoodInfoPOPULATION"]]
    .drop_duplicates()
    .set_index("NeighborhoodInfoNAME")["NeighborhoodInfoPOPULATION"]
    .to_dict()
)

# create description of logistic regression

mymarkdown='''This prediction tool uses a logistic regression to predict whether a reported crime is/ was/ will be a violent type of offense as opposed to other non-violent offense types. You can enter various values for the date, time, and neighborhood to receive various predictions. This model tested at 58% accuracy on the test set of the data. Please view the specifics of the model below the predictor tool.'''

odds_min = logreg_interpret_table["odds_ratio"].min()
odds_max = logreg_interpret_table["odds_ratio"].max()




In [50]:
import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point, Polygon
import geopy
import matplotlib as plt
import seaborn as sns
import pandas as pd
import contextily as cx
import numpy as np
from scipy.stats import pearsonr
import plotly.express as px
import plotly.graph_objects as go
import json
import glob
import os

In [51]:

crime_points=gpd.read_file("crime_points_neighborhood_.gpkg")
print(glob.glob("Police_Neighborhood_Area.*"))   # confirm which files exist

# allow GDAL to try to recreate the .shx
os.environ["SHAPE_RESTORE_SHX"] = "YES"
neighborhoods = gpd.read_file("Police_Neighborhood_Area.shp")
neighborhood_detail=pd.read_csv("Neighborhood_Detail_Records.csv")


['Police_Neighborhood_Area.shx', 'Police_Neighborhood_Area.shp', 'Police_Neighborhood_Area.cpg', 'Police_Neighborhood_Area.dbf', 'Police_Neighborhood_Area.prj']


In [52]:
crime_points=gpd.read_file("crime_points_neighborhood_.gpkg")
neighborhoods=gpd.read_file("Police_Neighborhood_Area.shp")
neighborhood_detail=pd.read_csv("Neighborhood_Detail_Records.csv")


# Develop a category map to assign each offense to a broader topical category to simplify and generalize analysis

category_map = {
    'Assault & Violent Crime': ['assault', 'battery', 'homicide', 'murder', 'rape', 'sex offense', 'threats', 'robbery','harassment','stalking','kidnap','prowler','extortion'],
    'Theft & Larceny': ['larceny', 'shoplifting', 'stolen', 'theft', 'embezzlement','burglary'],
    'Property Damage': ['vandalism', 'graffiti', 'damage', 'arson'],
    'Narcotics/Drug Offenses': ['narcotics', 'drug', 'marijuana', 'possession', 'distribute','driving under the influence','towed vehicle'],
    'Motor Vehicle/Traffic': ['traffic', 'dui', 'dwi', 'reckless', 'unlicensed', 'accident','hit and run'],
    'Weapon Offenses': ['weapon', 'firearm', 'gun', 'shots fired', 'illegal hunting'],
    'Public Order/Disorderly': ['disorderly', 'public intoxication', 'trespassing', 'loitering', 'noise','domestic disturbance','disturbarance - non domestic','trespass','prostitution'],
    'Fraud & Financial': ['fraud', 'forgery', 'counterfeit', 'scam', 'financial','false report of crime'],
    'Mental Health/Welfare': ['mental', 'tdo', 'eco', 'suicide', 'assist citizen','missing person','runaway','crisis assessment','juvenile investigation']
}

# Implement the category map to the crime data

def assign_category(offense_type):
    """
    Checks the offense type against defined keywords and assigns a category.
    Returns 'Other' if no match is found.
    """
    # Convert to lowercase for case-insensitive matching
    offense_lower = offense_type.lower()
    
    # Iterate through categories and their keywords
    for category, keywords in category_map.items():
        if any(keyword in offense_lower for keyword in keywords):
            return category
    
    return 'Other'


neighborhood_geo=pd.merge(neighborhoods,neighborhood_detail,how="left",left_on="NAME", right_on="NeighborhoodInfoNAME", validate="one_to_many")

neighborhood_geo['Offense_cat'] = neighborhood_geo['Offense'].apply(assign_category)

neighborhood_summary=neighborhood_geo.groupby(["NAME", "Offense_cat"]).agg({"RecordID":"count"}).reset_index()
neighborhood_summary=pd.merge(neighborhoods, neighborhood_summary, how="left", on="NAME", validate="one_to_many")
neighborhood_pivot = neighborhood_summary.pivot_table(
    index=["OBJECTID", "BEAT_NO", "NAME", "POPULATION", "DISTRICT", "GlobalID", "geometry"],
    columns="Offense_cat",
    values="RecordID",
    aggfunc="sum",   # sum counts if multiple rows per category
    fill_value=0     # fill missing combinations with 0
).reset_index()


crime_points['Offense_cat'] = crime_points['Offense'].apply(assign_category)
crime_points["Date"] = pd.to_datetime(crime_points["DateReport"])

crime_points=crime_points.to_crs("epsg:4326")

crime_points['lon'] = crime_points.geometry.x
crime_points['lat'] = crime_points.geometry.y

crime_points_25=crime_points[crime_points["Date"].dt.year.isin([2025])]
crime_points_25=crime_points_25[crime_points_25["RecordID"] != 2027]


neighborhood_pivot.columns.name = None
neighborhood_pivot = neighborhood_pivot.rename_axis(None, axis=1)
neighborhood_pivot_gdf = gpd.GeoDataFrame(neighborhood_pivot, geometry="geometry", crs=neighborhood_summary.crs)

geojson_str = neighborhood_pivot_gdf.to_json()

neighborhood_geojson = json.loads(geojson_str)




crime_points_25["Date"] = pd.to_datetime(crime_points_25["Date"])

# --- Dash app ---
app = dash.Dash(__name__)

# Unique offense categories and date range for dropdowns
offense_options = crime_points_25["Offense_cat"].sort_values().unique()
min_date = crime_points_25["Date"].min()
max_date = crime_points_25["Date"].max()

app.layout = html.Div([
    html.H1("Crime Map Viewer"),

    html.Div([
        html.Label("Select Offense Category:"),
        dcc.Dropdown(
            id="offense-dropdown",
            options=[{"label": x, "value": x} for x in offense_options],
            value="Narcotics/Drug Offenses",
            clearable=False
        ),
    ], style={"width": "48%", "display": "inline-block"}),

    dcc.Graph(id="crime-map", style={"height": "700px"})
])

# --- Callback to update map ---
@app.callback(
    Output("crime-map", "figure"),
    Input("offense-dropdown", "value")
)
def update_map(selected_offense):
    filtered_df = crime_points_25[
        crime_points_25["Offense_cat"] == selected_offense
    ]
    
    fig = go.Figure()
    fig.add_trace(
        go.Scattermapbox(
            lon=filtered_df["lon"],
            lat=filtered_df["lat"],
            mode="markers",
            marker=dict(size=6, color="red", opacity=0.7),
            text=filtered_df["Offense_cat"],
            hovertemplate="<b>%{text}</b><br>Lat: %{lat}<br>Lon: %{lon}<extra></extra>"
        )
    )
    
    fig.update_layout(
        mapbox_style="carto-positron",
        mapbox_zoom=12,
        mapbox_center={"lat": 38.03, "lon": -78.48},
        margin={"r":0,"t":0,"l":0,"b":0},
        showlegend=False
    )
    
    return fig

if __name__ == "__main__":
    app.run(debug=True)


crime_points=gpd.read_file("crime_points_neighborhood_.gpkg")
neighborhoods=gpd.read_file("Police_Neighborhood_Area.shp")
neighborhood_detail=pd.read_csv("Neighborhood_Detail_Records.csv")


# Develop a category map to assign each offense to a broader topical category to simplify and generalize analysis

category_map = {
    'Assault & Violent Crime': ['assault', 'battery', 'homicide', 'murder', 'rape', 'sex offense', 'threats', 'robbery','harassment','stalking','kidnap','prowler','extortion'],
    'Theft & Larceny': ['larceny', 'shoplifting', 'stolen', 'theft', 'embezzlement','burglary'],
    'Property Damage': ['vandalism', 'graffiti', 'damage', 'arson'],
    'Narcotics/Drug Offenses': ['narcotics', 'drug', 'marijuana', 'possession', 'distribute','driving under the influence','towed vehicle'],
    'Motor Vehicle/Traffic': ['traffic', 'dui', 'dwi', 'reckless', 'unlicensed', 'accident','hit and run'],
    'Weapon Offenses': ['weapon', 'firearm', 'gun', 'shots fired', 'illegal hunting'],
    'Public Order/Disorderly': ['disorderly', 'public intoxication', 'trespassing', 'loitering', 'noise','domestic disturbance','disturbarance - non domestic','trespass','prostitution'],
    'Fraud & Financial': ['fraud', 'forgery', 'counterfeit', 'scam', 'financial','false report of crime'],
    'Mental Health/Welfare': ['mental', 'tdo', 'eco', 'suicide', 'assist citizen','missing person','runaway','crisis assessment','juvenile investigation']
}

# Implement the category map to the crime data

def assign_category(offense_type):
    """
    Checks the offense type against defined keywords and assigns a category.
    Returns 'Other' if no match is found.
    """
    # Convert to lowercase for case-insensitive matching
    offense_lower = offense_type.lower()
    
    # Iterate through categories and their keywords
    for category, keywords in category_map.items():
        if any(keyword in offense_lower for keyword in keywords):
            return category
    
    return 'Other'


neighborhood_geo=pd.merge(neighborhoods,neighborhood_detail,how="left",left_on="NAME", right_on="NeighborhoodInfoNAME", validate="one_to_many")

neighborhood_geo['Offense_cat'] = neighborhood_geo['Offense'].apply(assign_category)

neighborhood_summary=neighborhood_geo.groupby(["NAME", "Offense_cat"]).agg({"RecordID":"count"}).reset_index()
neighborhood_summary=pd.merge(neighborhoods, neighborhood_summary, how="left", on="NAME", validate="one_to_many")
neighborhood_pivot = neighborhood_summary.pivot_table(
    index=["OBJECTID", "BEAT_NO", "NAME", "POPULATION", "DISTRICT", "GlobalID", "geometry"],
    columns="Offense_cat",
    values="RecordID",
    aggfunc="sum",   # sum counts if multiple rows per category
    fill_value=0     # fill missing combinations with 0
).reset_index()


crime_points['Offense_cat'] = crime_points['Offense'].apply(assign_category)
crime_points["Date"] = pd.to_datetime(crime_points["DateReport"])

crime_points=crime_points.to_crs("epsg:4326")

crime_points['lon'] = crime_points.geometry.x
crime_points['lat'] = crime_points.geometry.y

crime_points_25=crime_points[crime_points["Date"].dt.year.isin([2025])]
crime_points_25=crime_points_25[crime_points_25["RecordID"] != 2027]


neighborhood_pivot.columns.name = None
neighborhood_pivot = neighborhood_pivot.rename_axis(None, axis=1)
neighborhood_pivot_gdf = gpd.GeoDataFrame(neighborhood_pivot, geometry="geometry", crs=neighborhood_summary.crs)

geojson_str = neighborhood_pivot_gdf.to_json()

neighborhood_geojson = json.loads(geojson_str)




## KNN Section

In [53]:
fig_elbow = go.Figure()
fig_elbow.add_trace(go.Scatter(
    x=elbow_results["k"],
    y=elbow_results["error"],
    mode='lines+markers'
))
fig_elbow.update_layout(
    title="KNN Elbow Method",
    xaxis_title="K",
    yaxis_title="Error Rate",
    template="plotly_white"
)


In [54]:
fig_acc = go.Figure()
fig_acc.add_trace(go.Scatter(
    x=elbow_results["k"],
    y=elbow_results["accuracy"],
    mode="lines+markers"
))
fig_acc.update_layout(
    title="KNN Accuracy vs. K",
    xaxis_title="K",
    yaxis_title="Accuracy",
    template="plotly_white"
)


In [55]:
fig_perm = go.Figure()
fig_perm.add_trace(go.Bar(
    x=importance_df["Importance_Mean"],
    y=importance_df["Feature"],
    orientation='h'
))
fig_perm.update_layout(
    title="KNN Feature Importance (Permutation)",
    xaxis_title="Importance",
    yaxis_title="Feature",
    height=900,
    template="plotly_white"
)


In [56]:
category_labels = [
    'Assault & Violent Crime',
    'Fraud & Financial',
    'Mental Health/Welfare',
    'Motor Vehicle/Traffic',
    'Narcotics/Drug Offenses',
    'Other',
    'Property Damage',
    'Public Order/Disorderly',
    'Theft & Larceny',
    'Weapon Offenses'
]

In [57]:

fig_cm = px.imshow(
    cm_normalized,
    x=category_labels,
    y=category_labels,
    text_auto='.2f',  # show numbers as decimal
    aspect="auto",
    color_continuous_scale="pubugn",
    labels=dict(x="Predicted Label", y="Actual Label", color="Proportion")
)

fig_cm.update_layout( # Renamed fig.update_layout to fig_cm.update_layout
    xaxis=dict(tickangle=45),
    title="Confusion Matrix for KNN Model (Normalized by Row)"
)

In [58]:
crime = pd.read_csv("Crime_Data.csv")

crime["DateReported"] = pd.to_datetime(crime["DateReported"], errors="coerce")

# Turn HourReported like 1723 into 17
crime["Hour"] = (crime["HourReported"] // 100).astype("Int64")
crime["DayOfWeek"] = crime["DateReported"].dt.dayofweek
crime["Month"] = crime["DateReported"].dt.month

def uva_in_session(date):
    if pd.isna(date):
        return 0
    m = date.month
    d = date.day

    # Approx Spring: Jan 15 – May 10
    if (m == 1 and d >= 15) or (m in [2,3,4]) or (m == 5 and d <= 10):
        return 1

    # Approx Fall: Aug 20 – Dec 15
    if (m == 8 and d >= 20) or (m in [9,10,11]) or (m == 12 and d <= 15):
        return 1

    return 0

crime["InSession"] = crime["DateReported"].apply(uva_in_session)


In [59]:
crime_dummies = pd.get_dummies(crime["Offense_cat"], prefix="crime", drop_first=True)

X = pd.concat(
    [crime[["Hour", "DayOfWeek", "Month", "InSession"]], crime_dummies],
    axis=1
)


In [60]:
corr_X = X.corr()

fig_corr_X = px.imshow(
    corr_X,
    text_auto=True,
    color_continuous_scale="RdBu_r",
    aspect="auto",
    title="Correlation Matrix: Crime Variables, Time, and UVA Academic Calendar"
)

fig_corr_X.update_layout(width=1000, height=1000)

fig_corr_X.show()


In [61]:
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go

pca_pipe = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("pca", PCA())
])

pca_pipe.fit(X)

pca_model = pca_pipe.named_steps["pca"]
explained_var = pca_model.explained_variance_ratio_
cum_explained_var = np.cumsum(explained_var)
n_components = len(explained_var)


In [62]:
ev_df = pd.DataFrame({
    "PC": np.arange(1, n_components + 1),
    "ExplainedVariance": explained_var,
    "CumulativeVariance": cum_explained_var
})

fig_scree = px.line(
    ev_df,
    x="PC",
    y="ExplainedVariance",
    markers=True,
    title="Scree Plot: Proportion of Variance Explained"
)

fig_scree.show()


In [63]:
X_pca_scores = pca_pipe.transform(X)
pc_cols = [f"PC{i}" for i in range(1, n_components + 1)]

scores_df = pd.DataFrame(X_pca_scores, columns=pc_cols)
scores_df["Offense_cat"] = crime["Offense_cat"]
scores_df["InSession"] = crime["InSession"]

fig_scatter = px.scatter(
    scores_df,
    x="PC1",
    y="PC2",
    color="InSession",
    hover_data=["Offense_cat"],
    title="Crimes in PC1–PC2 Space (Colored by UVA In-Session Flag)"
)

fig_scatter.show()


In [64]:
loadings = pca_model.components_.T[:, :2]

loading_df = pd.DataFrame(
    loadings,
    index=X.columns,
    columns=["PC1_loading", "PC2_loading"]
)

arrow_scale = 3
fig = go.Figure()

# points
fig.add_trace(go.Scatter(
    x=scores_df["PC1"],
    y=scores_df["PC2"],
    mode="markers",
    text=scores_df["Offense_cat"],
    name="Crimes",
    hoverinfo="text"
))

# arrows
for var_name, row in loading_df.iterrows():
    fig.add_trace(go.Scatter(
        x=[0, row["PC1_loading"] * arrow_scale],
        y=[0, row["PC2_loading"] * arrow_scale],
        mode="lines+text",
        text=[None, var_name],
        textposition="top center",
        showlegend=False
    ))

fig.update_layout(
    title="PCA Biplot: PC1 vs PC2 with Crime + Time + UVA Calendar Loadings",
    xaxis_title="PC1",
    yaxis_title="PC2",
    width=900,
    height=800,
    xaxis=dict(zeroline=True),
    yaxis=dict(zeroline=True),
)

fig.show()


In [65]:
tab_knn = dcc.Tab(label="KNN Model", children=[
    html.Br(),
    html.H3("KNN Classification Model"),

    dash_table.DataTable(
        data=[
            {"Metric": "Best K (Elbow Method)", "Value": knn_summary["best_k_elbow"]},
            {"Metric": "Best GridSearch K", "Value": knn_summary["best_k_grid"]},
            {"Metric": "Cross-Validation Accuracy", "Value": f"{knn_summary['cv_accuracy']:.4f}"},
            {"Metric": "Test Set Accuracy", "Value": f"{knn_summary['test_accuracy']:.4f}"}
        ],
        columns=[{"name": "Metric", "id": "Metric"}, {"name": "Value", "id": "Value"}],
        style_cell={'textAlign': 'left', 'padding': '6px'},
        style_header={'fontWeight': 'bold'},
        style_table={'width': '55%'}
    ),

    html.Br(),
    html.H4("Elbow Curve"),
    dcc.Graph(figure=fig_elbow),

    html.Br(),
    html.H4("Accuracy vs. K"),
    dcc.Graph(figure=fig_acc),

    html.Br(),
    html.H4("Permutation Feature Importance"),
    dcc.Graph(figure=fig_perm),

    html.Br(),
    html.H4("Confusion Matrix"),
    dcc.Graph(figure=fig_cm)
])


In [73]:
app = dash.Dash(__name__)

app.layout = html.Div(
    style={'font-family': 'Arial, sans-serif', 'padding': '20px'},
    children=[

        # ---------------- TITLE ----------------
        html.H1(
            "A Geographic & Predictive Analysis of Crime In Charlottesville",
            style={'textAlign': 'center', 'marginBottom': '30px'}
        ),

        # ---------------- TABS ----------------
        dcc.Tabs([

            # ----- TAB 1: Overview & Context -----
            dcc.Tab(label="Overview & Context", children=[
                html.Br(),

                html.H3("Project Overview"),
                html.P("""
                    This dashboard provides a geographic and predictive analysis of crime in Charlottesville. 
                    Tab one summarizes contextual information, key data characteristics, and descriptive metrics. 
                    Subsequent tabs provide insights into machine learning models applied for predictive analytics.
                """),
                html.Br(),
                html.H3("Interactive Crime Map"),
                html.Div([
                    html.Label("Select Offense Category:"),
                    dcc.Dropdown(
                        id="offense-dropdown",
                        options=[{"label": x, "value": x} for x in crime_points_25["Offense_cat"].sort_values().unique()],
                        value="Narcotics/Drug Offenses",
                        clearable=False
                    ),
                ], style={"width": "48%", "display": "inline-block"}),

                dcc.Graph(id="crime-map", style={"height": "700px"})

            ]),
            dcc.Tab(label='Linear Regression & Plots', children=[
                html.Div([
                    html.H3("OLS Regression Summary"),
                    html.Pre(summary_text, style={"whiteSpace": "pre-wrap"}),

                    html.H3("Crime Count vs Avg Temp Scatter Plot with OLS Trendline"),
                    dcc.Graph(figure=fig_scatter_avg_temp),

                    html.H3("Crime Count vs Precipitation Scatter Plot with OLS Trendline"),
                    dcc.Graph(figure=fig_scatter_prcp),

                    html.H3("Crime Count vs Wind Speed Scatter Plot with OLS Trendline"),
                    dcc.Graph(figure=fig_scatter_wind),
                ])
            ]),
            # Insert this definition right after the odds_max calculation or before the main app definition

            dcc.Tab(label='Logistic Regression', children=[
                html.Div([
                    html.H2("Generate a Prediction of Crime Violence"),
                    dcc.Markdown(children=mymarkdown),

                    # Prediction Inputs
                    dcc.Input(id='year', type='number', placeholder='Year', style={'marginRight': '5px'}),
                    dcc.Input(id='month', type='number', placeholder='Month', style={'marginRight': '5px'}),
                    dcc.Input(id='day', type='number', placeholder='Day', style={'marginRight': '5px'}),
                    dcc.Input(id='hour', type='number', placeholder='Hour', style={'marginRight': '5px'}),

                    dcc.Dropdown(
                        id='neighborhood',
                        options=[{'label': n, 'value': n} for n in X_log['NeighborhoodInfoNAME'].unique()],
                        placeholder="Neighborhood",
                        style={'width': '200px', 'display': 'inline-block', 'marginRight': '10px'}
                    ),

                    html.Button("Predict", id='predict-button', n_clicks=0),

                    html.Br(),
                    html.Div(id='prediction-output', style={'marginTop': '10px', 'fontWeight': 'bold'}),

                    html.H2("Logistic Regression Model Results"),

                    # Odds Ratio Table
                    dash_table.DataTable(
                        id='logreg-results',
                        columns=[{"name": col, "id": col} for col in logreg_interpret_table.columns],
                        data=logreg_interpret_table.to_dict('records'),
                        sort_action="native",
                        page_size=15,
                        style_table={'overflowX': 'scroll', 'maxWidth': '100%'},
                        style_cell={
                            'textAlign': 'left', 'padding': '6px', 'minWidth': '120px', 'whiteSpace': 'normal'
                        },
                        style_header={'fontWeight': 'bold'},
                        
                        # Conditional Formatting
                        style_data_conditional=[
                            {
                                "if": {"filter_query": f"{{odds_ratio}} >= {odds_min}"},
                                "backgroundColor": "rgb(255, 240, 240)",
                            },
                            {
                                "if": {"filter_query": f"{{odds_ratio}} >= {odds_min + 0.25*(odds_max-odds_min)}"},
                                "backgroundColor": "rgb(255, 200, 200)",
                            },
                            {
                                "if": {"filter_query": f"{{odds_ratio}} >= {odds_min + 0.50*(odds_max-odds_min)}"},
                                "backgroundColor": "rgb(255, 150, 150)",
                            },
                            {
                                "if": {"filter_query": f"{{odds_ratio}} >= {odds_min + 0.75*(odds_max-odds_min)}"},
                                "backgroundColor": "rgb(255, 100, 100)",
                            },
                            {
                                "if": {"filter_query": f"{{odds_ratio}} >= {odds_max}"},
                                "backgroundColor": "rgb(255, 50, 50)",
                                "color": "white"
                            }
                        ]
                    )
                ])
            ]),


            dcc.Tab(label='KMeans Analysis', children=[
                html.Div([
                    html.H3("Clustered Offense Categories"),
                    dcc.Graph(figure=fig_bar),

                    html.H3("Boxplot of tavg by Cluster"),
                    dcc.Graph(figure=fig_box_tavg),

                    html.H3("Tukey’s HSD Results"),
                    html.Pre(tukey_summary, style={"whiteSpace": "pre-wrap"}),

                    html.H3("Boxplot of HourReported by Cluster"),
                    dcc.Graph(figure=fig_box_hour),

                    html.H3("Boxplot of prcp by Cluster"),
                    dcc.Graph(figure=fig_box_prcp)
                    
                ])
            ]),
            dcc.Tab(label='KNN Analysis', children=[
                html.Div([
                    html.H3("KNN Best Model Summary"),
                    dash_table.DataTable(
                        # Manual data construction using values from your screenshots
                        data=[
                            {"Metric": "Best Parameters", "Value": "21, 1, distance"},
                            {"Metric": "Best CV Accuracy", "Value": "0.2956"},
                            {"Metric": "Test Set Accuracy", "Value": "0.2966"}
                        ],
                        columns=[
                            {"name": "Metric", "id": "Metric"},
                            {"name": "Value", "id": "Value"}
                        ],
                        style_table={'width': '50%'},
                        style_cell={'textAlign': 'left', 'padding': '8px'},
                        style_header={'fontWeight': 'bold'}
                    ),
                    html.H3("Elbow Method Results for KNN"),
                    dcc.Graph(figure=fig_elbow),

                    html.H3("Model Accuracy vs. Number of Neighbors"),
                    dcc.Graph(figure=fig_acc),

                    html.H3("Permutation Feature Importance for KNN Model"),
                    dcc.Graph(figure=fig_perm),

                    html.H3("Confusion Matrix for KNN Model"),
                    dcc.Graph(figure=fig_cm)
                    
                ])
            ]),
            dcc.Tab(label='PCA Analysis', children=[
                html.Div([
                    html.H3("PCA Correlation Matrix"),
                    dcc.Graph(figure=fig_corr_X)
                    ,
                    html.H3("PCA Proportion of Variance Explained"),
                    dcc.Graph(figure=fig_scree),

                    html.H3("Crimes in PC1-PC2 Space Colored by UVA In-Session Flag"),
                    dcc.Graph(figure=fig_scatter),

                    html.H3("PCA Biplot: PC1 vs PC2 with Crime + Time + UVA Calendar"),
                    dcc.Graph(figure=fig),
                    
                ])
            ])

        ])
    ]
)

@app.callback(
    Output("crime-map", "figure"),
    Input("offense-dropdown", "value")
)
def update_map(selected_offense):
    filtered_df = crime_points_25[
        crime_points_25["Offense_cat"] == selected_offense
    ]
    
    fig = go.Figure()
    fig.add_trace(
        go.Scattermapbox(
            lon=filtered_df["lon"],
            lat=filtered_df["lat"],
            mode="markers",
            marker=dict(size=6, color="red", opacity=0.7),
            text=filtered_df["Offense_cat"],
            hovertemplate="<b>%{text}</b><br>Lat: %{lat}<br>Lon: %{lon}<extra></extra>"
        )
    )
    
    fig.update_layout(
        mapbox_style="carto-positron",
        mapbox_zoom=12,
        mapbox_center={"lat": 38.03, "lon": -78.48},
        margin={"r":0,"t":0,"l":0,"b":0},
        showlegend=False
    )
    
    return fig

if __name__ == '__main__':
    app.run(debug=True)

/var/folders/09/89r30_ms6xdcd7znk61plqgr0000gn/T/ipykernel_14833/1863047557.py:208: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [ ]:
print(neighborhoods.columns.tolist())


In [74]:
!pip install streamlit plotly geopandas scikit-learn statsmodels


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 48.2 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 75.5 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [streamlit]10 [streamlit]


In [76]:
import pandas as pd

# List of CSV files to trim
files = [
    "complete_data.csv",
    "knn_elbow_results.csv",
    "knn_feature_importance.csv",
    "knn_best.csv",
    "CrimeData_with_Neighborhood_Info.csv",
    "Neighborhood_Detail_Records.csv",
    "Crime_Data.csv"
]

for file in files:
    df = pd.read_csv(file)
    # Take first 5000 rows, or sample randomly if needed
    if len(df) > 5000:
        df = df.sample(5000, random_state=42).reset_index(drop=True)
    df.to_csv(f"/Users/yani/Desktop/ML_Project_Yani/ML_Project_Crime_Data/ML_Project_Crime_Data_GitHub/data/{file}", index=False)


In [77]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json

# --- Example for complete_data ---
complete_data.head(5000).to_csv("complete_data_csv.csv", index=False)

# --- Example for KNN CSVs ---
pd.read_csv("knn_elbow_results.csv").head(5000).to_csv("knn_elbow_results_csv.csv", index=False)
pd.read_csv("knn_feature_importance.csv").head(5000).to_csv("knn_feature_importance_csv.csv", index=False)
pd.read_csv("knn_best.csv").head(5000).to_csv("knn_best_csv.csv", index=False)

# --- Convert GeoPackages and Shapefiles to CSV ---
crime_points = gpd.read_file("crime_points_neighborhood_.gpkg")
crime_points.head(5000).to_csv("crime_points_csv.csv", index=False)

neighborhoods = gpd.read_file("Police_Neighborhood_Area.shp")
neighborhoods.head(5000).to_csv("neighborhoods_csv.csv", index=False)

neighborhood_detail = pd.read_csv("Neighborhood_Detail_Records.csv")
neighborhood_detail.head(5000).to_csv("neighborhood_detail_csv.csv", index=False)

# --- Optional JSON to CSV ---
try:
    with open("knn_summary.json") as f:
        knn_summary = json.load(f)
    pd.DataFrame([knn_summary]).to_csv("knn_summary_csv.csv", index=False)
except FileNotFoundError:
    print("knn_summary.json not found")
